#CelebAMask Lips

##Imports

In [1]:
!python --version

Python 3.9.16


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [3]:
!tar -xzvf "/content/drive/MyDrive/Human_Parsing_Dataset/CIHP/instance-level_human_parsing.tar.gz" -C "/content/"

Streaming output truncated to the last 5000 lines.
instance-level_human_parsing/Testing/Images/0026811.jpg
instance-level_human_parsing/Testing/Images/0037573.jpg
instance-level_human_parsing/Testing/Images/0033068.jpg
instance-level_human_parsing/Testing/Images/0035298.jpg
instance-level_human_parsing/Testing/Images/0002339.jpg
instance-level_human_parsing/Testing/Images/0017562.jpg
instance-level_human_parsing/Testing/Images/0037435.jpg
instance-level_human_parsing/Testing/Images/0004422.jpg
instance-level_human_parsing/Testing/Images/0011872.jpg
instance-level_human_parsing/Testing/Images/0007900.jpg
instance-level_human_parsing/Testing/Images/0015033.jpg
instance-level_human_parsing/Testing/Images/0036605.jpg
instance-level_human_parsing/Testing/Images/0028815.jpg
instance-level_human_parsing/Testing/Images/0020045.jpg
instance-level_human_parsing/Testing/Images/0037771.jpg
instance-level_human_parsing/Testing/Images/0033402.jpg
instance-level_human_parsing/Testing/Images/0036409.j

# Create Face Directory

In [4]:
import os

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Create Directory for Flickr dataset
create_dir("/content/Face_Dataset")
create_dir("/content/Face_Dataset/original")
create_dir("/content/Face_Dataset/mask")
create_dir("/content/Face_Dataset/segmentation")
create_dir("/content/Face_Dataset/tfrecord")
create_dir("/content/Face_Dataset/ImageSets")

# Initialize

In [5]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import cv2
from glob import glob
original_images = sorted(glob("/content/instance-level_human_parsing/Validation/Images/*"))
mask_images = sorted(glob("/content/instance-level_human_parsing/Validation/Category_ids/*"))
print( len( original_images ) , len( mask_images))
print( original_images[100], mask_images[100])
check_shape = cv2.imread(mask_images[100])
print("Image shape:", check_shape.shape)

5000 5000
/content/instance-level_human_parsing/Validation/Images/0001120.jpg /content/instance-level_human_parsing/Validation/Category_ids/0001120.png
Image shape: (333, 500, 3)


In [7]:
unique_value_dic = {}
for i in check_shape:
  for ii in i:
    for iii in ii:
      if str(iii) in unique_value_dic:
        unique_value_dic[str(iii)] += 1
      else:
        unique_value_dic[str(iii)] = 1
print(unique_value_dic)

{'0': 287928, '2': 35295, '13': 29235, '5': 168210, '10': 660, '15': 9198, '14': 9474}


In [12]:
# CELEB A MASK
from tqdm import tqdm
import tensorflow as tf
import cv2

# Initialize stats
# fail_count = 0
# success_count = 0
# savedImages_count = 0

mask_loc = "/content/instance-level_human_parsing/Validation/Category_ids/"

# Initialize where the generated dataset will be stored
NEW_PATH = "/content/Face_Dataset/"
NEW_ORIGINAL_PATH = "/content/Face_Dataset/original/"
NEW_MASK_PATH = "/content/Face_Dataset/mask/"

# Just change the number of iteration depending on the size of your initial repository
for i_p in tqdm(original_images):
    try:
        # Read and resize the image using TensorFlow
        img = tf.io.read_file(i_p)
        img = tf.io.decode_png(img, channels=3)
        img = tf.image.resize(img, (257, 257), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        read_image = tf.cast(img, tf.uint8).numpy()

        cut1 = i_p.split("/")
        cut2 = cut1[-1].split(".")
        fileName = cut2[0]

        # Read and resize the mask using TensorFlow
        mask = tf.io.read_file(mask_loc + fileName + ".png")
        mask = tf.io.decode_png(mask, channels=3)  
        mask = tf.image.resize(mask, (257, 257), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        read_mask = tf.cast(mask, tf.uint8).numpy()

        # Convert image from RGB to BGR
        read_image = cv2.cvtColor(read_image, cv2.COLOR_RGB2BGR)

        # Save the resized image and mask
        cv2.imwrite(NEW_ORIGINAL_PATH + "image_{}.jpg".format(savedImages_count), read_image)
        cv2.imwrite(NEW_MASK_PATH + "image_{}.png".format(savedImages_count), read_mask)
        savedImages_count += 1

        success_count += 1
    except:
        fail_count += 1


100%|██████████| 5000/5000 [01:33<00:00, 53.55it/s]


In [13]:
# Check stats
print( fail_count )
print( success_count )
print( savedImages_count)

0
33280
33280


In [14]:

maskCount = sorted(glob( "/content/Face_Dataset/mask/*" ))

originalCount = sorted(glob( "/content/Face_Dataset/original/*" ))

print( len(maskCount), len(originalCount))

33280 33280


# Change python interpreter (Working)

In [18]:
#install python 3.7 and dev utils
#you may not need all the dev libraries, but I haven't tested which aren't necessary.
!sudo apt-get update -y
!sudo apt-get install python3.7 python3.7-dev python3.7-distutils libpython3.7-dev

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 2

# install pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall

#install colab's dependencies
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

# link to the old google package
!ln -s /usr/local/lib/python3.9/dist-packages/google \
       /usr/local/lib/python3.7/dist-packages/google

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [77.6 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,010 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,052 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,726 kB]
Get:13 http://se

In [19]:
#check python version
import sys
print(sys.version)
!python3 --version
!python --version

3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
Python 3.7.16
Python 3.7.16


In [20]:
!python --version
#Python 3.7.15

Python 3.7.16


In [21]:
!pip --version

pip 23.1.2 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [22]:
!pip uninstall traitlets
!pip install traitlets==5.7.1

Found existing installation: traitlets 5.9.0
Uninstalling traitlets-5.9.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/traitlets-5.9.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/traitlets/*
Proceed (Y/n)? y
  Successfully uninstalled traitlets-5.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.9/109.9 kB 4.4 MB/s eta 0:00:00


In [23]:
!pip install --upgrade ipython
!pip install --upgrade ipykernel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
# !pip install -q --upgrade ipython
# !pip install -q --upgrade ipykernel

In [25]:
!python --version

Python 3.7.16


# Installs

In [1]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install matplotlib
!pip install opencv-python

!pip install tf-slim==1.0
!pip install tensorflow==1.15

# Install the segmentation model of pytorch
!pip install segmentation-models-pytorch

# Install kaggle to google colab
!pip install -q kaggle

!pip install glob2

# Install these if it is not yet installed
!pip install numpy
!pip install mediapipe
!pip install importlib-metadata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 54.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 16.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 28.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [3]:
# Import necessary models
import os
import time

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import cv2


# Make kaggle directory
!mkdir /kaggle

# Mount the google drive
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
# Import tensorflow
import tensorflow as tf
import os

from IPython.display import clear_output
import matplotlib.pyplot as plt
from glob import glob
from pathlib import Path


# Imports
import cv2
import numpy as np
from time import time
import matplotlib.pyplot as plt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


# Create Tfrecord

In [1]:
!python --version

Python 3.7.16


In [4]:
tf.__version__

'1.15.0'

In [5]:
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

env: PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


In [6]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 85664, done.
remote: Counting objects: 100% (1263/1263), done.
remote: Compressing objects: 100% (518/518), done.
remote: Total 85664 (delta 795), reused 1170 (delta 739), pack-reused 84401
Receiving objects: 100% (85664/85664), 598.68 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (61302/61302), done.


In [8]:
import os

import tensorflow as tf
from PIL import Image
from tqdm import tqdm
import numpy as np

import os, shutil

In [9]:
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
import numpy as np

import os, shutil

# 0 - Background
# 1 - Hair
# 2 - Face
# 3 - Top-Body-Skin
# 4 - Top-Body-Clothes
# 5 - Lower-Body-Skin
# 6 - Lower-Body-Clothes
# 7 - Upper-Accessories
# 8 - Lower-Accessories
# palette (color map) describes the (R, G, B): Label pair
palette = {(0,   0,   0) : 0 ,
         (1,  1, 1) : 7,
         (2,  2, 2) : 1,
         (3,  3, 3) : 7,
         (4,  4, 4) : 7,
         (5,  5, 5) : 4,
         (6,  6, 6) : 4,
         (7,  7, 7) : 4,
         (8,  8, 8) : 8,
         (9,  9, 9) : 6,
         (10,  10, 10) : 3,
         (11,  11, 11) : 7,
         (12,  12, 12) : 4,
         (13,  13, 13) : 2,
         (14,  14, 14) : 3,
         (15,  15, 15) : 3,
         (16,  16, 16) : 5,
         (17,  17, 17) : 5,
         (18,  18, 18) : 8,
         (19,  19, 19) : 8
         }

def convert_from_color_segmentation(arr_3d):
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)

    for c, i in palette.items():
        m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
        arr_2d[m] = i
    return arr_2d


label_dir = '/content/Face_Dataset/mask/'
new_label_dir = '/content/Face_Dataset/segmentation/'

if not os.path.isdir(new_label_dir):
	print("creating folder: ",new_label_dir)
	os.mkdir(new_label_dir)
else:
	print("Folder alread exists. Delete the folder and re-run the code!!!")


label_files = os.listdir(label_dir)

count = 1
train_list = []
val_list = []
trainval_list = []

for l_f in tqdm(label_files):
    arr = np.array(Image.open(label_dir + l_f))
    arr = arr[:,:,0:3]
    arr_2d = convert_from_color_segmentation(arr)
    Image.fromarray(arr_2d).save(new_label_dir + l_f)

    #print( "New label: ", new_label_dir)
    #print( "LF: ", l_f)
    test = l_f.split(".")
    test2 = str(test[0]).split("_")
    imageNum = test2[-1]
    #print( "TEST: ", test)
    #print( "New lf: ", l_f)
    trainval_list.append("image_" + imageNum)
    if count > 8:
      val_list.append("image_" + imageNum)
      count += 1
    if count <= 8:
      train_list.append("image_" + imageNum)
      count += 1
    if count == 11:
      count = 1


"""
count = 1
train_list = []
val_list = []
trainval_list = []

for l_f in tqdm(label_files):
    arr = np.array(Image.open(label_dir + l_f))
    arr = arr[:,:,0:3]
    arr_2d = convert_from_color_segmentation(arr)
    #print( "New label: ", new_label_dir)
    #print( "LF: ", l_f)
    test = l_f.split(".")
    test2 = str(test[0]).split("_")
    imageNum = test2[-1]
    l_f = "original_" + imageNum + ".jpg"
    #print( "TEST: ", test)
    #print( "New lf: ", l_f)
    trainval_list.append("original_" + imageNum)
    if count > 8:
      val_list.append("original_" + imageNum)
      count += 1
    if count <= 8:
      train_list.append("original_" + imageNum)
      count += 1
    if count == 10:
      count = 1
    tempImage = Image.fromarray(arr_2d)
    tempImage = tempImage.resize((513, 513), Image.ANTIALIAS )
    tempImage.save(new_label_dir + l_f)
"""

Folder alread exists. Delete the folder and re-run the code!!!


100%|██████████| 33280/33280 [34:25<00:00, 16.11it/s]


'\ncount = 1\ntrain_list = []\nval_list = []\ntrainval_list = []\n\nfor l_f in tqdm(label_files):\n    arr = np.array(Image.open(label_dir + l_f))\n    arr = arr[:,:,0:3]\n    arr_2d = convert_from_color_segmentation(arr)\n    #print( "New label: ", new_label_dir)\n    #print( "LF: ", l_f)\n    test = l_f.split(".")\n    test2 = str(test[0]).split("_")\n    imageNum = test2[-1]\n    l_f = "original_" + imageNum + ".jpg"\n    #print( "TEST: ", test)\n    #print( "New lf: ", l_f)\n    trainval_list.append("original_" + imageNum)\n    if count > 8:\n      val_list.append("original_" + imageNum)\n      count += 1\n    if count <= 8:\n      train_list.append("original_" + imageNum)\n      count += 1\n    if count == 10:\n      count = 1\n    tempImage = Image.fromarray(arr_2d)\n    tempImage = tempImage.resize((513, 513), Image.ANTIALIAS )\n    tempImage.save(new_label_dir + l_f)\n'

In [10]:
print( len (train_list))
print( len(val_list))
print( len(trainval_list))
print( count)

26624
6656
33280
1


In [11]:
# define list of places
with open( "/content/Face_Dataset/ImageSets/train.txt" , 'w') as filehandle:
  for listitem in train_list:
    filehandle.write('%s\n' % listitem)
with open( "/content/Face_Dataset/ImageSets/trainval.txt" , 'w') as filehandle:
  for listitem in trainval_list:
    filehandle.write('%s\n' % listitem)
with open( "/content/Face_Dataset/ImageSets/val.txt" , 'w') as filehandle:
  for listitem in val_list:
    filehandle.write('%s\n' % listitem)

In [15]:
# Merge Mask
import cv2

# Load the image
image_path = "/content/Face_Dataset/segmentation/image_100.png"
image = cv2.imread(image_path)

# Check image properties
height, width, channels = image.shape
image_type = image.dtype

print("Image: ", image)
print("Image path:", image_path)
print("Image shape:", image.shape)
print("Image height:", height)
print("Image width:", width)
print("Number of channels:", channels)
print("Image data type:", image_type)


# Display the image
# cv2.imshow("Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Image:  [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
Image path: /content/Face_Dataset/segmentation/image_100.png
Image shape: (257, 257, 3)
Image height: 257
Image width: 257
Number of channels: 3
Image data type: uint8


In [16]:
unique = []
for i in image:
  for ii in i:
    for iii in ii:
      if iii in unique:
        pass
      else:
        unique.append(iii)
        print(iii)

0
1
2
4
3


In [17]:
tf.__version__

'1.15.0'

In [18]:
!python --version

Python 3.7.16


In [ ]:
!pip install tensorflow==1.15

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.13.0rc0)
ERROR: No matching distribution found for tensorflow==1.15


In [19]:
%cd /content/models/research/deeplab/datasets

/content/models/research/deeplab/datasets


In [20]:
# Lint as: python2, python3
# Copyright 2018 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Converts PASCAL VOC 2012 data to TFRecord file format with Example protos.

PASCAL VOC 2012 dataset is expected to have the following directory structure:

  + pascal_voc_seg
    - build_data.py
    - build_voc2012_data.py (current working directory).
    + VOCdevkit
      + VOC2012
        + JPEGImages
        + SegmentationClass
        + ImageSets
          + Segmentation
    + tfrecord

Image folder:
  ./VOCdevkit/VOC2012/JPEGImages

Semantic segmentation annotations:
  ./VOCdevkit/VOC2012/SegmentationClass

list folder:
  ./VOCdevkit/VOC2012/ImageSets/Segmentation

This script converts data into sharded data files and save at tfrecord folder.

The Example proto contains the following fields:

  image/encoded: encoded image content.
  image/filename: image filename.
  image/format: image file format.
  image/height: image height.
  image/width: image width.
  image/channels: image channels.
  image/segmentation/class/encoded: encoded semantic segmentation content.
  image/segmentation/class/format: semantic segmentation file format.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import math
import os.path
import sys
import build_data
from six.moves import range
import tensorflow as tf

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('image_folder',
                           '/content/Face_Dataset/original',
                           'Folder containing images.')

tf.app.flags.DEFINE_string(
    'semantic_segmentation_folder',
    '/content/Face_Dataset/segmentation',
    'Folder containing semantic segmentation annotations.')

tf.app.flags.DEFINE_string(
    'list_folder',
    '/content/Face_Dataset/ImageSets',
    'Folder containing lists for training and validation')

tf.app.flags.DEFINE_string(
    'output_dir',
    '/content/Face_Dataset/tfrecord',
    'Path to save converted SSTable of TensorFlow examples.')


_NUM_SHARDS = 100


def _convert_dataset(dataset_split):
  """Converts the specified dataset split to TFRecord format.

  Args:
    dataset_split: The dataset split (e.g., train, test).

  Raises:
    RuntimeError: If loaded image and label have different shape.
  """
  dataset = os.path.basename(dataset_split)[:-4]
  sys.stdout.write('Processing ' + dataset)
  filenames = [x.strip('\n') for x in open(dataset_split, 'r')]
  num_images = len(filenames)
  num_per_shard = int(math.ceil(num_images / _NUM_SHARDS))

  image_reader = build_data.ImageReader('jpeg', channels=3)
  label_reader = build_data.ImageReader('png', channels=1)

  for shard_id in range(_NUM_SHARDS):
    output_filename = os.path.join(
        FLAGS.output_dir,
        '%s-%05d-of-%05d.tfrecord' % (dataset, shard_id, _NUM_SHARDS))
    with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
      start_idx = shard_id * num_per_shard
      end_idx = min((shard_id + 1) * num_per_shard, num_images)
      for i in range(start_idx, end_idx):
        sys.stdout.write('\r>> Converting image %d/%d shard %d' % (
            i + 1, len(filenames), shard_id))
        sys.stdout.flush()
        # Read the image.
        image_filename = os.path.join(
            #MH:
            #FLAGS.image_folder, filenames[i] + '.' + FLAGS.image_format)
            FLAGS.image_folder, filenames[i] + '.jpg')
            #END MH
        image_data = tf.gfile.GFile(image_filename, 'rb').read()
        height, width = image_reader.read_image_dims(image_data)
        # Read the semantic segmentation annotation.
        seg_filename = os.path.join(
            FLAGS.semantic_segmentation_folder,
            #MH:
            #filenames[i] + '.' + FLAGS.label_format)
            filenames[i] + '.png')
            #END MH
        seg_data = tf.gfile.GFile(seg_filename, 'rb').read()
        seg_height, seg_width = label_reader.read_image_dims(seg_data)
        if height != seg_height or width != seg_width:
          raise RuntimeError('Shape mismatched between image and label.')
        # Convert to tf example.
        example = build_data.image_seg_to_tfexample(
            image_data, filenames[i], height, width, seg_data)
        tfrecord_writer.write(example.SerializeToString())
    sys.stdout.write('\n')
    sys.stdout.flush()


def main(unused_argv):
  dataset_splits = tf.gfile.Glob(os.path.join(FLAGS.list_folder, '*.txt'))
  for dataset_split in dataset_splits:
    _convert_dataset(dataset_split)


if __name__ == '__main__':
  tf.app.run()


W0529 14:48:54.454324 139630213232448 module_wrapper.py:139] From /tmp/ipykernel_9466/2941631966.py:146: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0529 14:48:54.484401 139630213232448 module_wrapper.py:139] From /content/models/research/deeplab/datasets/build_data.py:63: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0529 14:48:54.511844 139630213232448 module_wrapper.py:139] From /content/models/research/deeplab/datasets/build_data.py:65: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2023-05-29 14:48:54.528007: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.7/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-29 14:48:54.528073: E tensorflow/stream_executor/cuda/cud

>> Converting image 1/33280 shard 0

W0529 14:48:54.659685 139630213232448 module_wrapper.py:139] From /tmp/ipykernel_9466/2941631966.py:124: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



>> Converting image 333/33280 shard 0
>> Converting image 666/33280 shard 1
>> Converting image 999/33280 shard 2
>> Converting image 1332/33280 shard 3
>> Converting image 1665/33280 shard 4
>> Converting image 1998/33280 shard 5
>> Converting image 2331/33280 shard 6
>> Converting image 2664/33280 shard 7
>> Converting image 2997/33280 shard 8
>> Converting image 3330/33280 shard 9
>> Converting image 3663/33280 shard 10
>> Converting image 3996/33280 shard 11
>> Converting image 4329/33280 shard 12
>> Converting image 4662/33280 shard 13
>> Converting image 4995/33280 shard 14
>> Converting image 5328/33280 shard 15
>> Converting image 5661/33280 shard 16
>> Converting image 5994/33280 shard 17
>> Converting image 6327/33280 shard 18
>> Converting image 6660/33280 shard 19
>> Converting image 6993/33280 shard 20
>> Converting image 7326/33280 shard 21
>> Converting image 7659/33280 shard 22
>> Converting image 7992/33280 shard 23
>> Converting image 8325/33280 shard 24
>> Converting

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!cp -r /content/Lips_Dataset/ImageSets /content/drive/MyDrive/Lips_170k

In [ ]:
!pwd

In [22]:
%cd /content/

/content


In [ ]:
!zip -r /content/drive/MyDrive/Lips_Tfrecord.zip Lips_Tfrecord/

In [24]:
import shutil

In [25]:
shutil.move("/content/Face_Dataset/tfrecord", "/content/drive/MyDrive/CIHP_Dataset_257") 

'/content/drive/MyDrive/CIHP_Dataset_257/tfrecord'

In [ ]:
drive.flush_and_unmount()

In [18]:
import shutil
shutil.rmtree('/content/Face_Dataset/segmentation')

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive
